# JSON to Parquet


I'm kinda just playing around with different datafiles here


## 00 Imports


In [1]:
# %load_ext autoreload
# %autoreload 2

In [5]:
import json
from pathlib import Path

import orjson
import jsonlines
from loguru import logger
import polars as pl

In [6]:
__file__ = __vsc_ipynb_file__
PROJ_ROOT = Path(__file__).parent.parent
logger.info(PROJ_ROOT)
DATA_DIR = PROJ_ROOT / "data"
logger.info(DATA_DIR)
MLP_MSE_VALIDATION = DATA_DIR / "mlp_mse_validation"
MLP_MSE_VALIDATION_TENSORS = MLP_MSE_VALIDATION / "validation_mses.safetensors"
MLP_MSE_VALIDATION_META = MLP_MSE_VALIDATION / "validation_mses.json"

2025-01-07 20:03:46.958 | INFO     | __main__:<cell line: 0>:3 - /Users/gat/work/FA2024/embedding_translation
2025-01-07 20:03:46.959 | INFO     | __main__:<cell line: 0>:5 - /Users/gat/work/FA2024/embedding_translation/data


In [10]:
with MLP_MSE_VALIDATION_META.open() as f:
    meta = orjson.loads(f.read())

meta

{'dataset2idx': {'arguana': 0,
  'fiqa': 1,
  'scidocs': 2,
  'nfcorpus': 3,
  'hotpotqa': 4,
  'trec-covid': 5},
 'model2idx': {'WhereIsAI_UAE-Large-V1': 0,
  'BAAI_bge-base-en-v1.5': 1,
  'BAAI_bge-large-en-v1.5': 2,
  'BAAI_bge-small-en-v1.5': 3,
  'intfloat_e5-base-v2': 4,
  'intfloat_e5-large-v2': 5,
  'intfloat_e5-small-v2': 6,
  'thenlper_gte-base': 7,
  'thenlper_gte-large': 8,
  'thenlper_gte-small': 9,
  'sentence-transformers_gtr-t5-base': 10,
  'sentence-transformers_gtr-t5-large': 11,
  'mixedbread-ai_mxbai-embed-large-v1': 12,
  'sentence-transformers_sentence-t5-base': 13,
  'sentence-transformers_sentence-t5-large': 14,
  'openai_text-embedding-3-large': 15,
  'openai_text-embedding-3-small': 16},
 'num_layers': [2, 3, 4, 5, 6, 7]}

In [4]:
data_dir = PROJ_ROOT / "data"
output_dir = data_dir / "01_interim"
output_dir.mkdir(exist_ok=True)

## 01 Load


In [5]:
def load_stitch_pairs(file_path: Path) -> pl.DataFrame:
    """Load stitch pair information into a DataFrame."""
    with Path.open(file_path) as f:
        data = json.load(f)
    return pl.DataFrame(
        [
            {
                "source_model": data["source"],
                "target_model": data["target"],
                "dataset": data["dataset"],
                "stitch_mode": data["mode"],
            },
        ],
    )

In [6]:
stitch_pairs = load_stitch_pairs(
    data_dir
    / "arguana_loss"
    / "BAAI_bge-base-en-v1.5_BAAI_bge-large-en-v1.5/arguana/stitch_info_pairs.json",
)

stitch_pairs.write_parquet(output_dir / "stitch_pairs.parquet")
stitch_pairs

source_model,target_model,dataset,stitch_mode
str,str,str,str
"""BAAI/bge-base-en-v1.5""","""BAAI/bge-large-en-v1.5""","""arguana""","""affine"""
